In [1]:
import numpy as np

In [3]:
# Defines the biological network, Markov Decision Process and helper functions 

# We have 4 genes: [ATM, p53, Wip1, MDM2].
# Each gene can be 0 (OFF) or 1 (ON).
# 16 possible states, each is a 4D bit vector.

def build_states():
    """
    Returns a list of 16 states, each a 4-bit vector [b0,b1,b2,b3].
    Also returns two dicts:
      state_to_index[tuple_bits] = index in [0..15]
      index_to_state[index] = tuple_bits
    """
    states = []
    for i in range(16):
        bits = [(i >> b) & 1 for b in reversed(range(4))]  # [ATM,p53,Wip1,MDM2]
        states.append(bits)
    state_to_index = {}
    index_to_state = {}
    for idx, s_bits in enumerate(states):
        t = tuple(s_bits)
        state_to_index[t] = idx
        index_to_state[idx] = t
    return states, state_to_index, index_to_state

# The action space A = {a1= [0,0,0,0], a2= [1,0,0,0], a3= [0,1,0,0], a4= [0,0,1,0], a5= [0,0,0,1]}
    
def build_actions():
    """
    Returns a list of 5 actions (4D bit vectors), plus two dicts for index <-> action.
    We'll map action indices to the labels a1, a2, a3, a4, a5 for printing.
    """
    actions = [
        [0,0,0,0],  # a1
        [1,0,0,0],  # a2
        [0,1,0,0],  # a3
        [0,0,1,0],  # a4
        [0,0,0,1],  # a5
    ]
    action_to_index = {}
    # Create label dictionary: 0->"a1", 1->"a2", etc.
    index_to_action = {0: "a1", 1: "a2", 2: "a3", 3: "a4", 4: "a5"}

    for i, a_bits in enumerate(actions):
        t = tuple(a_bits)
        action_to_index[t] = i

    return actions, action_to_index, index_to_action

# Connectivity matrix C (from the figure)

C = np.array([
    [ 0,  0, -1,  0],
    [ 1,  0, -1, -1],
    [ 0,  1,  0,  0],
    [-1,  1,  1,  0]
], dtype=int)

def threshold_and_convert(vec):
    """
    For a 4D integer vector 'vec', apply:
      if > 0 => 1, else => 0
    Returns a 4D bit vector in {0,1}^4.
    """
    out = []
    for x in vec:
        if x > 0:
            out.append(1)
        else:
            out.append(0)
    return out

def xor_bits(vecA, vecB):
    """
    Component-wise XOR for two 4D vectors in {0,1}.
    Returns a new list of length 4.
    """
    return [(a ^ b) for (a,b) in zip(vecA, vecB)]

def hamming_distance(vecA, vecB):
    """
    Return the number of components where vecA differs from vecB.
    Both are length-4 bit vectors.
    """
    return sum(a != b for a,b in zip(vecA, vecB))

In [5]:
# Function to define and build the transition model and reward model 

def build_transition_matrices(states, state_to_index, actions, p):
    """
    Build a list of 5 transition matrices M(a), each shape (16,16).
    M(a)[i,j] = Probability of going from state i to j if we pick action a.
    
    The formula from the problem is:
       M(a)_{i,j} = p^d (1-p)^(4-d)
    where d = HammingDistance( o_j ,  C*o_i (thresholded) XOR a ).
    """
    num_states = 16
    num_actions = 5
    M = np.zeros((num_actions, num_states, num_states), dtype=float)

    for a_idx, a_bits in enumerate(actions):
        for i in range(num_states):
            o_i = states[i]  # e.g. [0,1,0,1]
            # 1) Compute C*o_i (as an integer vector), then threshold
            ci = C.dot(o_i)  # shape (4,)
            ci_thresh = threshold_and_convert(ci)  # in {0,1}^4
            # 2) XOR with a
            x = xor_bits(ci_thresh, a_bits)
            
            # For each possible next state j:
            for j in range(num_states):
                o_j = states[j]
                d = hamming_distance(o_j, x)
                prob = (p**d)*((1.0 - p)**(4-d))
                M[a_idx, i, j] = prob
    return M

def build_reward_array(states, actions):
    """
    R[s,a,s'] = 5*(#ON in s') - |a|.
    where |a| is the sum of bits in action a.
    """
    num_states = 16
    num_actions = 5
    R = np.zeros((num_actions, num_states, num_states), dtype=float)
    for a_idx, a_bits in enumerate(actions):
        cost_a = sum(a_bits)  # e.g. 0 or 1
        for i in range(num_states):
            for j in range(num_states):
                o_j = states[j]
                num_on = sum(o_j)
                R[a_idx, i, j] = 5*num_on - cost_a
    return R

In [7]:
# Value Iteration Technique

def value_iteration(M, R, gamma=0.95, theta=0.01):
    """
    M: shape (5,16,16) => M[a,i,j]
    R: shape (5,16,16) => R[a,i,j]
    gamma: discount factor
    theta: convergence threshold

    Returns: (V, pi, num_iterations)
      V: shape (16,) final value function
      pi: shape (16,) optimal action index for each state
      num_iterations: how many iterations until convergence
    """
    num_actions, num_states, _ = M.shape
    V = np.zeros(num_states)
    iteration = 0

    while True:
        iteration += 1
        V_new = np.zeros(num_states)
        for i in range(num_states):
            best_val = -1e9
            # Check each action
            for a_idx in range(num_actions):
                Qa = 0.0
                for j in range(num_states):
                    prob = M[a_idx, i, j]
                    r = R[a_idx, i, j]
                    Qa += prob * (r + gamma * V[j])
                if Qa > best_val:
                    best_val = Qa
            V_new[i] = best_val
        delta = np.max(np.abs(V_new - V))
        V = V_new
        if delta < theta:
            break

    # Extract policy
    pi = np.zeros(num_states, dtype=int)
    for i in range(num_states):
        best_a = 0
        best_val = -1e9
        for a_idx in range(num_actions):
            Qa = 0.0
            for j in range(num_states):
                prob = M[a_idx, i, j]
                r = R[a_idx, i, j]
                Qa += prob * (r + gamma * V[j])
            if Qa > best_val:
                best_val = Qa
                best_a = a_idx
        pi[i] = best_a

    return V, pi, iteration

In [9]:
# Policy Iteration Technique

def policy_evaluation(pi, M, R, gamma=0.95, theta=0.01):
    """
    Evaluate a fixed policy pi, shape (16,).
    M[a,i,j], R[a,i,j].
    Returns: V (shape (16,))
    """
    num_states = M.shape[1]
    V = np.zeros(num_states)
    while True:
        V_new = np.zeros(num_states)
        for i in range(num_states):
            a_idx = pi[i]
            val = 0.0
            for j in range(num_states):
                prob = M[a_idx, i, j]
                r = R[a_idx, i, j]
                val += prob * (r + gamma*V[j])
            V_new[i] = val
        delta = np.max(np.abs(V_new - V))
        V = V_new
        if delta < theta:
            break
    return V

def policy_improvement(V, M, R, gamma=0.95):
    """
    Given V, compute pi_new by greedy improvement.
    """
    num_actions, num_states, _ = M.shape
    pi_new = np.zeros(num_states, dtype=int)
    for i in range(num_states):
        best_a = 0
        best_val = -1e9
        for a_idx in range(num_actions):
            Qa = 0.0
            for j in range(num_states):
                prob = M[a_idx, i, j]
                r = R[a_idx, i, j]
                Qa += prob*(r + gamma*V[j])
            if Qa > best_val:
                best_val = Qa
                best_a = a_idx
        pi_new[i] = best_a
    return pi_new

def policy_iteration(M, R, gamma=0.95, theta=0.01):
    """
    Returns: (V, pi, iteration_count)
    """
    num_states = M.shape[1]
    pi = np.zeros(num_states, dtype=int)  # start with a1 in all states
    iteration = 0
    while True:
        iteration += 1
        V = policy_evaluation(pi, M, R, gamma, theta)
        pi_new = policy_improvement(V, M, R, gamma)
        if np.array_equal(pi_new, pi):
            break
        pi = pi_new
    return V, pi, iteration

In [11]:
# Functions to simulate the policy to measure average activation of the genes in the biological network

def simulate_policy(pi, M, states, p, num_episodes=100, episode_length=200):
    """
    Sample multiple episodes from the MDP using a fixed policy pi.
    Return average number of ON genes (AvgA).
    """
    num_states = 16
    A_list = []
    for _ in range(num_episodes):
        s = np.random.randint(0, num_states)
        total_on = 0
        for step in range(episode_length):
            o_bits = states[s]
            total_on += sum(o_bits)
            a_idx = pi[s]
            probs = M[a_idx, s, :]
            s_next = np.random.choice(num_states, p=probs)
            s = s_next
        A_i = total_on / float(episode_length)
        A_list.append(A_i)
    return np.mean(A_list)

def simulate_no_control(M, states, p, num_episodes=100, episode_length=200):
    """
    "No control" => always action a1 => index=0.
    So the transition matrix is M[0].
    """
    num_states = 16
    A_list = []
    for _ in range(num_episodes):
        s = np.random.randint(0, num_states)
        total_on = 0
        for step in range(episode_length):
            o_bits = states[s]
            total_on += sum(o_bits)
            probs = M[0, s, :]
            s_next = np.random.choice(num_states, p=probs)
            s = s_next
        A_i = total_on / float(episode_length)
        A_list.append(A_i)
    return np.mean(A_list)

In [13]:
# Implement value iteration and policy iteration for given scenarios

def main():
    # Build states & actions
    states, state_to_index, index_to_state = build_states()
    actions, action_to_index, index_to_action = build_actions()

    # Define a helper to run Value Iteration for a given p, then test the resulting policy.
    def run_value_iteration_for_p(p):
        # Build M(a), R(a)
        M = build_transition_matrices(states, state_to_index, actions, p)
        R = build_reward_array(states, actions)
        
        # Run value iteration
        V, pi, iters = value_iteration(M, R, gamma=0.95, theta=0.01)
        
        # Convert numeric pi to labeled actions (e.g. "a1", "a2", etc.)
        labeled_pi = [f"a{a_idx+1}" for a_idx in pi]

        # Compare with no control
        avgA_noctrl = simulate_no_control(M, states, p)
        
        # Compare with optimal policy
        avgA_opt = simulate_policy(pi, M, states, p)

        print(f"\nValue Iteration for p={p}:")
        print(f" - Converged in {iters} iterations.")
        print(" - Final Value Function (16 states) =", np.round(V,1))
        print(" - Final Policy (16 states) =", labeled_pi)
        print(f" - Avg Activation no control = {avgA_noctrl:.2f}")
        print(f" - Avg Activation optimal    = {avgA_opt:.2f}")

    # Part (a) => p=0.04
    run_value_iteration_for_p(0.04)

    # Part (b) => p=0.15, p=0.48
    run_value_iteration_for_p(0.15)
    run_value_iteration_for_p(0.48)

    # Part (c) => p=0.05, do Policy Iteration (starting with a1 in all states)
    
    p_c = 0.05
    M_c = build_transition_matrices(states, state_to_index, actions, p_c)
    R_c = build_reward_array(states, actions)
    V_pi, pi_pi, pi_iters = policy_iteration(M_c, R_c, gamma=0.95, theta=0.01)
    
    # Also compare no control vs. policy iteration result
    labeled_pi_pi = [f"a{a_idx+1}" for a_idx in pi_pi]
    avgA_noctrl_c = simulate_no_control(M_c, states, p_c)
    avgA_opt_c = simulate_policy(pi_pi, M_c, states, p_c)

    print(f"\nPolicy Iteration for p={p_c}:")
    print(f" - Converged in {pi_iters} policy improvements.")
    print(" - Final Value Function (16 states) =", np.round(V_pi,1))
    print(" - Final Policy (16 states) =", labeled_pi_pi)
    print(f" - Avg Activation no control = {avgA_noctrl_c:.2f}")
    print(f" - Avg Activation optimal    = {avgA_opt_c:.2f}")

    # Compare with the Value Iteration result for p=0.05,
    print("\n(Compare with Value Iteration at p=0.05):")
    run_value_iteration_for_p(0.05)

if __name__ == "__main__":
    main()


Value Iteration for p=0.04:
 - Converged in 142 iterations.
 - Final Value Function (16 states) = [258.4 258.4 262.8 262.8 267.7 267.7 267.7 267.7 263.1 258.4 258.4 258.4
 267.7 263.1 267.7 267.7]
 - Final Policy (16 states) = ['a3', 'a3', 'a3', 'a3', 'a3', 'a3', 'a3', 'a3', 'a4', 'a3', 'a3', 'a3', 'a5', 'a3', 'a3', 'a3']
 - Avg Activation no control = 0.42
 - Avg Activation optimal    = 2.87

Value Iteration for p=0.15:
 - Converged in 139 iterations.
 - Final Value Function (16 states) = [227.6 227.6 230.8 230.8 234.8 234.8 234.8 234.8 231.4 227.6 227.6 227.6
 234.8 231.4 234.8 234.8]
 - Final Policy (16 states) = ['a3', 'a3', 'a3', 'a3', 'a3', 'a3', 'a3', 'a3', 'a4', 'a3', 'a3', 'a3', 'a5', 'a3', 'a3', 'a3']
 - Avg Activation no control = 1.04
 - Avg Activation optimal    = 2.54

Value Iteration for p=0.48:
 - Converged in 136 iterations.
 - Final Value Function (16 states) = [196.  196.  196.2 196.2 196.4 196.4 196.4 196.4 196.2 196.  196.  196.
 196.4 196.2 196.4 196.4]
 - Final 